In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import warnings
from datetime import datetime
import pandas 
import pandas_datareader as pdr
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
start_date = datetime(1990, 1, 1)
end_date = datetime(2020, 8, 31)
spx = pdr.get_data_yahoo(symbols='^SP500TR', start=start_date, end=end_date)["Adj Close"]
vix = pdr.get_data_yahoo(symbols='^vix', start=start_date, end=end_date)["Adj Close"]
bnp = pandas.concat([spx, vix], axis=1)
#Update Here! update column name base on features
bnp.columns = ('spx','vix')
bnp = bnp.sort_values(by="Date",ascending=True)
print(bnp.head(10))
bnp.to_csv('Result.csv')

#看資料內容
bnp.head()

                   spx        vix
Date                             
1990-01-02  386.160004  17.240000
1990-01-03  385.170013  18.190001
1990-01-04  382.019989  19.219999
1990-01-05  378.299988  20.110001
1990-01-08  380.040009  20.260000
1990-01-09  375.559998  22.200001
1990-01-10  373.089996  22.440001
1990-01-11  374.399994  20.049999
1990-01-12  365.220001  24.639999
1990-01-15  362.079987  26.340000


,spx,vix
Date,,
1990-01-02,386.160004,17.240000
1990-01-03,385.170013,18.190001
1990-01-04,382.019989,19.219999
1990-01-05,378.299988,20.110001
1990-01-08,380.040009,20.260000


In [3]:
print('資料维度:', bnp.shape)

資料维度: (7727, 2)


In [4]:

bnp = pandas.get_dummies(bnp)
bnp.head(5)

,spx,vix
Date,,
1990-01-02,386.160004,17.240000
1990-01-03,385.170013,18.190001
1990-01-04,382.019989,19.219999
1990-01-05,378.299988,20.110001
1990-01-08,380.040009,20.260000


In [5]:
# 答案
labels = np.array(bnp['spx'])

# 將答案移除
bnp= bnp.drop('spx', axis = 1)

# 名字单独保存一下，以备后患
bnp_list = list(bnp.columns)

# 轉成np array格式
bnp = np.array(bnp)

In [6]:
bnp.shape

(7727, 1)

In [7]:
from sklearn import preprocessing
input_features = preprocessing.StandardScaler().fit_transform(bnp)

In [8]:
input_features[0]

array([-0.26619062])

### 建立模型

In [ ]:
x = torch.tensor(input_features, dtype = float)

y = torch.tensor(labels, dtype = float)

# 權重參數
weights = torch.randn((1, 7727), dtype = float, requires_grad = True) 
biases = torch.randn(7727, dtype = float, requires_grad = True) 
weights2 = torch.randn((7727, 1), dtype = float, requires_grad = True) 
biases2 = torch.randn(1, dtype = float, requires_grad = True) 

learning_rate = 0.001 
losses = []

for i in range(1000):
    # 计算隐层
    hidden = x.mm(weights) + biases
    # 加入激活函数
    hidden = torch.relu(hidden)
    # 预测结果
    predictions = hidden.mm(weights2) + biases2
    # 算損失
    loss = torch.mean((predictions - y) ** 2) 
    losses.append(loss.data.numpy())
    
    # 打印损失值
    if i % 100 == 0:
        print('loss:', loss)
    #返向传播计算
    loss.backward()
    
    #更新参数
    weights.data.add_(- learning_rate * weights.grad.data)  
    biases.data.add_(- learning_rate * biases.grad.data)
    weights2.data.add_(- learning_rate * weights2.grad.data)
    biases2.data.add_(- learning_rate * biases2.grad.data)
    
    # 每次迭代都得记得清空
    weights.grad.data.zero_()
    biases.grad.data.zero_()
    weights2.grad.data.zero_()
    biases2.grad.data.zero_()



loss: tensor(7073520.8520, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(nan, dtype=torch.float64, grad_fn=<MeanBackward0>)


In [ ]:
predictions.shape

### 更简单的构建网络模型

In [ ]:
input_size = input_features.shape[1]
hidden_size = 128
output_size = 1
batch_size = 16
my_nn = torch.nn.Sequential(
    torch.nn.Linear(input_size, hidden_size),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size, output_size),
)
cost = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(my_nn.parameters(), lr = 0.001)

In [ ]:
# 训练网络
losses = []
for i in range(1000):
    batch_loss = []
    # MINI-Batch方法来进行训练
    for start in range(0, len(input_features), batch_size):
        end = start + batch_size if start + batch_size < len(input_features) else len(input_features)
        xx = torch.tensor(input_features[start:end], dtype = torch.float, requires_grad = True)
        yy = torch.tensor(labels[start:end], dtype = torch.float, requires_grad = True)
        prediction = my_nn(xx)
        loss = cost(prediction, yy)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        batch_loss.append(loss.data.numpy())
    
    # 打印损失
    if i % 100==0:
        losses.append(np.mean(batch_loss))
        print(i, np.mean(batch_loss))

预测训练结果

In [60]:
x = torch.tensor(input_features, dtype = torch.float)
predict = my_nn(x).data.numpy()

In [ ]:
# 转换日期格式
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# 创建一个表格来存日期和其对应的标签数值
true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})

# 同理，再创建一个来存日期和其对应的模型预测值
months = features[:, feature_list.index('month')]
days = features[:, feature_list.index('day')]
years = features[:, feature_list.index('year')]

test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]

test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]

predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': predict.reshape(-1)}) 

In [ ]:
# 真实值
plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')

# 预测值
plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 
plt.legend()

# 图名
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual and Predicted Values');
